## Initiate the Toolbox
- the parameters set here will be used to collect command-specific data when the `Fetcher` executes the TET pattern.

In [38]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbol=["AAPL"],
    interval="1d",
    start_date="2021-01-01",
    end_date="2021-01-01",
)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
result = toolbox.technical.mandelbrot_channel(window="1m")

In [40]:
result

date,symbol,bottom_price,recent_price,top_price
date,str,f64,f64,f64
2024-03-18,"""AAPL""",171.0914,175.482803,176.9097


In [20]:
from humbldata.core.standard_models.toolbox import ToolboxQueryParams


toolbox2 = ToolboxQueryParams(symbol=["AAPL", "MSFT"])

